# Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from startup import np, pd, plt, sns

In [3]:
from startup_gui import widgets, interact, interactive

In [4]:
import tensorflow as tf
import tensorflow_probability as tfp

2023-09-17 10:27:32.090 Python[38529:1014855] apply_selection_policy_once: avoid use of removable GPUs (via org.python.python:GPUSelectionPolicy->avoidRemovable)


In [5]:
tfd = tfp.distributions
tfb = tfp.bijectors

In [107]:
from journal_20230625_pgm_functions import (
    sat, intelligence, grade, difficulty, letter,
    p_d, p_i, p_s_i, p_g_id, p_l_g,
)

# Joint Distribution with Vector Events

In [6]:
x = tfd.Bernoulli(probs=0.5)

In [8]:
y = tfd.Bernoulli(probs=0.8)

In [9]:
n_samples = 10_000
df = pd.DataFrame({'x': x.sample(n_samples).numpy(), 'y': y.sample(n_samples)})

In [14]:
# _ = df.hist()

In [96]:
def dist_to_df(joint, n_samples):
    return pd.DataFrame(dict(zip(['x', 'y'], joint.sample(n_samples))))

In [18]:
joint = tfd.JointDistributionSequential(
    [
        x, 
        y
    ]
)

In [98]:
dist_to_df(joint, n_samples).describe()

,x,y
count,10000.000000,10000.000000
mean,0.506100,0.805300
std,0.499988,0.395989
min,0.000000,0.000000
25%,0.000000,1.000000
50%,1.000000,1.000000
75%,1.000000,1.000000
max,1.000000,1.000000


In [24]:
joint.log_prob([[0, 0], [0, 1]])

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([-2.3025851, -0.9162907], dtype=float32)>

In [32]:
joint.sample([5, 2])

[<tf.Tensor: shape=(5, 2), dtype=int32, numpy=
 array([[0, 0],
        [1, 1],
        [0, 1],
        [1, 0],
        [0, 1]], dtype=int32)>,
 <tf.Tensor: shape=(5, 2), dtype=int32, numpy=
 array([[1, 0],
        [0, 1],
        [0, 0],
        [1, 1],
        [0, 1]], dtype=int32)>]

## Second distribution depends on first


In [49]:
joint_2 = tfd.JointDistributionSequential(
    [
        x, 
        lambda e: tfd.Bernoulli(probs=tf.where(e == 1, 0.5, 0.8))
    ],
    batch_ndims=0
)

In [100]:
dist_to_df(joint_2, n_samples).describe()

,x,y
count,10000.000000,10000.000000
mean,0.495800,0.641800
std,0.500007,0.479495
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,1.000000
75%,1.000000,1.000000
max,1.000000,1.000000


In [101]:
dist_to_df(joint_2, n_samples).groupby('x').agg(['mean', 'std'])

y          
       mean       std
x                    
0  0.798227  0.401363
1  0.506802  0.500004

In [104]:
joint_3 = tfd.JointDistributionSequential(
    [
        x, 
        lambda e: tfd.Categorical(probs=tf.gather(tf.constant([[0.2, 0.8], [0.5, 0.5]]), e))
    ],
    batch_ndims=0
)

In [105]:
dist_to_df(joint_3, n_samples).groupby('x').agg(['mean', 'std'])

y          
       mean       std
x                    
0  0.804243  0.396822
1  0.493405  0.500006

In [106]:
joint_3.sample([5, 2])

[<tf.Tensor: shape=(5, 2), dtype=int32, numpy=
 array([[1, 0],
        [1, 0],
        [0, 1],
        [1, 0],
        [0, 0]], dtype=int32)>,
 <tf.Tensor: shape=(5, 2), dtype=int32, numpy=
 array([[1, 1],
        [1, 1],
        [1, 0],
        [1, 1],
        [1, 1]], dtype=int32)>]

In [108]:
p_d

,prob
difficulty,
easy,0.6
hard,0.4


In [121]:
tf.gather(tf.gather(tf.constant(p_g_id.values.reshape([ len(grade), len(intelligence), len(difficulty),])), 0, axis=1), 1, axis=1)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([0.05, 0.25, 0.7 ])>